# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure the API Key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('..\Output_Data\Weather_Data_Collection.csv')
weather_df = weather_df.drop(columns={'Unnamed: 0'})
weather_df.head()



,City,Lattitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,portland,45.52,-122.68,20.00,84,0,1.18,US,1601529367
1,hobart,-42.88,147.33,15.00,38,75,4.60,AU,1601529298
2,chapais,49.78,-74.85,7.00,100,90,2.10,CA,1601529581
3,kavarskas,55.43,24.92,9.16,74,100,4.92,LT,1601529581
4,fairbanks,64.84,-147.72,13.00,43,1,3.10,US,1601529313


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Obtaining the Coordinates to plot on the map
city_locations = weather_df[['Lattitude','Longitude']].astype(float)
city_locations.head()

,Lattitude,Longitude
0,45.52,-122.68
1,-42.88,147.33
2,49.78,-74.85
3,55.43,24.92
4,64.84,-147.72


In [4]:
# Humidity values
humidity_data = weather_df['Humidity']
humidity_data.head()

0     84
1     38
2    100
3     74
4     43
Name: Humidity, dtype: int64

In [5]:
# Creating a Humidity Heatmap Layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(city_locations, weights=humidity_data, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(humidity_layer)

fig



Figure(layout=FigureLayout(height=&#39;420px&#39;))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# The most ideal vacations involve destinations with minimal cloud cover as well as having temperatures within the range of 20-30 degrees celcius. As such, there datframe will be narrowed down further in order to find hotels within these regions that have the ideal weather. 

ideal_weather_df = weather_df.loc[
    (weather_df['Cloudiness']==0) & 
    (weather_df['Wind Speed'] <= 10) & 
    (weather_df['Max Temp'] >= 20) &
    (weather_df['Max Temp'] <= 30)].dropna()

print(f' The Number of Ideal Cities within the ideal weather criteria are {len(ideal_weather_df)}')
ideal_weather_df


 The Number of Ideal Cities within the ideal weather criteria are 18


,City,Lattitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,portland,45.52,-122.68,20.00,84,0,1.18,US,1601529367
78,caravelas,-17.71,-39.25,21.99,91,0,2.25,BR,1601529628
99,carnarvon,-24.87,113.63,23.00,64,0,9.30,AU,1601529417
103,morondava,-20.28,44.28,26.53,55,0,1.24,MG,1601529642
112,elat,29.56,34.95,27.00,61,0,5.10,IL,1601529557
128,jumla,29.27,82.18,23.30,34,0,2.07,NP,1601529657
145,sonoita,31.85,-112.83,28.89,15,0,1.55,MX,1601529667
235,mareeba,-17.00,145.43,29.44,48,0,6.70,AU,1601529723
338,flinders,-34.58,150.86,24.44,35,0,3.58,AU,1601529789
351,abu samrah,24.60,72.70,24.82,61,0,2.47,IN,1601529797


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Creating a New Hotel DataFram
hotel_df = ideal_weather_df.copy().drop(columns={'Date', 'Wind Speed','Max Temp','Humidity','Cloudiness'})

hotel_df['Hotel Name'] = ""

hotel_df


,City,Lattitude,Longitude,Country,Hotel Name
0,portland,45.52,-122.68,US,
78,caravelas,-17.71,-39.25,BR,
99,carnarvon,-24.87,113.63,AU,
103,morondava,-20.28,44.28,MG,
112,elat,29.56,34.95,IL,
128,jumla,29.27,82.18,NP,
145,sonoita,31.85,-112.83,MX,
235,mareeba,-17.00,145.43,AU,
338,flinders,-34.58,150.86,AU,
351,abu samrah,24.60,72.70,IN,


In [8]:
# Setting up the parameters to determine the closest hotels within a 5 km radium (5000 m)

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Setting up the search parameters
params = {
    'type':'hotel',
    'keyword': 'hotel',
    'radius': 5000,
    'key': g_key
}

# We will be itterating through the rows in order to hotels within the defined radius of the coordinates of each city
print(f'Data retrival in progress')
for index, row in hotel_df.iterrows():
    latitude = row['Lattitude']
    longitude = row['Longitude']
    city = row['City']

    # Now that the latitude and longitude has been defined, these need to be added to the dictionary

    params['location'] = f'{latitude},{longitude}'

    
    response = requests.get(base_url, params=params).json()

    search_results = response['results']

    try:
        print(f'Finding nearest hotel for {city}\n--------------------------')
        hotel_df.loc[index, 'Hotel Name'] = search_results[0]['name']
    
    except (KeyError, IndexError):
        print(f'There are no Hotels for {city}, moving on to the next city....\n')


print('End of Data Retreival\n*********************')
print(f'Removing cities where hotels could not be retreived')
hotel_df = hotel_df[hotel_df['Hotel Name']!=""]




Data retrival in progress
Finding nearest hotel for portland
--------------------------
Finding nearest hotel for caravelas
--------------------------
Finding nearest hotel for carnarvon
--------------------------
Finding nearest hotel for morondava
--------------------------
Finding nearest hotel for elat
--------------------------
Finding nearest hotel for jumla
--------------------------
Finding nearest hotel for sonoita
--------------------------
Finding nearest hotel for mareeba
--------------------------
Finding nearest hotel for flinders
--------------------------
Finding nearest hotel for abu samrah
--------------------------
Finding nearest hotel for kiama
--------------------------
Finding nearest hotel for bowen
--------------------------
Finding nearest hotel for aden
--------------------------
Finding nearest hotel for sonqor
--------------------------
Finding nearest hotel for abyek
--------------------------
Finding nearest hotel for agadez
--------------------------
The

In [9]:
# Results of the Hotel Search
hotel_df

,City,Lattitude,Longitude,Country,Hotel Name
0,portland,45.52,-122.68,US,Kimpton RiverPlace Hotel
78,caravelas,-17.71,-39.25,BR,Mar Doce de Abrolhos
99,carnarvon,-24.87,113.63,AU,Carnarvon Motel
103,morondava,-20.28,44.28,MG,Palissandre Cote Ouest resort & SPA
112,elat,29.56,34.95,IL,"Al Manara, a Luxury Collection Hotel, Saraya A..."
128,jumla,29.27,82.18,NP,Shrestha Hotel
145,sonoita,31.85,-112.83,MX,Hotel Excelsior
235,mareeba,-17.00,145.43,AU,Taj on Byrnes Mareeba Luxury Accommodation
338,flinders,-34.58,150.86,AU,Lakeview Hotel Motel
351,abu samrah,24.60,72.70,IN,Sterling Mount Abu


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lattitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
hotel_marker = gmaps.marker_layer(locations, info_box_content=hotel_info)

# We will add the new layer to the previous humidity heatmap
fig.add_layer(hotel_marker)

# Display figure
fig

Figure(layout=FigureLayout(height=&#39;420px&#39;))